<a href="https://colab.research.google.com/github/legobitna/DeepLearning-basic/blob/main/Question_4_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    # DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000

    sentences = []
    labels = []
    # YOUR CODE HERE
    # How To read the json file 
    with open("sarcasm.json", 'r') as f:
      datastore = json.load(f)
    
    # Seperate the data and label 
    for item in datastore:
      sentences.append(item['headline'])
      labels.append(item['is_sarcastic'])
    
    # seperate to train and test set 
    training_sentences = sentences[0:training_size]
    testing_sentences = sentences[training_size:]
    training_labels = labels[0:training_size]
    testing_labels = labels[training_size:]

    # Change to array 
    training_labels = np.array(training_labels)
    testing_labels = np.array(testing_labels)

    # TOKENIZER 셋업 
    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
    tokenizer.fit_on_texts(training_sentences)

    # 글자에 인덱스 부여 해서 나열, 자주쓰는게 앞에옴 
    word_index = tokenizer.word_index

    # 내가 가지고있는 데이터에 인덱스 부여 
    training_sequences = tokenizer.texts_to_sequences(training_sentences)

    # 문장마다 사이즈가 다르기 떄문에 패딩을 넣어준다 
    training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # 테스트 데이터에도 같은 작업  데이터에 인덱스부여 -> 패딩부여 
    testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # 모델만들기 
    model = tf.keras.Sequential([
    # YOUR CODE HERE. KEEP THIS OUTPUT LAYER INTACT OR TESTS MAY FAIL
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # SARCASM 인지 아닌지 두가지로만 나타내면 되니까 
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()

    num_epochs = 10

    # 트레이닝 x랑 , 트레이닝 Y랑 ,epochs사이즈랑, 밸리데이션 데이터 세트 (x,y)
    history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 64)           12544     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 54,961
Trainable params: 54,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 70s 112ms/step - loss: 0.4433 - accuracy: 0.7746 - val_loss: 0.3